In [1]:
from urllib.request import urlopen
import json
import pandas as pd

In [ ]:
in_file = r'../data files/HGNC_gene_info.tsv'
out_filename = r'../data files/HPO_diseases.tsv'

csv_data = pd.read_csv(in_file, sep="\t")
data = []
row_num = 0
for index, row in csv_data.iterrows():
    try:
        row_num += 1
        hgnc_id = csv_data["HGNC_ID"][index]
        ncbi_id = csv_data["NCBI_ID"][index]
        gene_symbol = csv_data["Approved_symbol"][index]
        # print("processing: ", ncbi_id)
        url = 'https://hpo.jax.org/api/hpo/gene/' + str(int(ncbi_id))
        # print("url: ", url)
        response = urlopen(url)

        data_json = json.loads(response.read())
        
        for disease in data_json["diseaseAssoc"]:
            data.append({"HGNC_ID": hgnc_id, "Gene_symbol": gene_symbol, "Disease_ID": disease["diseaseId"], "Disease_name": disease["diseaseName"]})

    except:
        print('Failure on row ', row_num)

if row_num % 100 == 0:
    print("Rows processed: ", row_num)
    df = pd.DataFrame(data)
    df.to_csv(out_filename, sep = '\t', index=False, mode='a', header=False)
    data = []

df = pd.DataFrame(data)
df.to_csv(out_filename, sep = '\t', index=False, mode='a', header=False)

In [ ]:
# The last step is to open the generated file and fill in the HGNC_ID column based on the Gene_symbol
df = pd.read_csv(out_filename, sep='\t')

# Fill in the HGNC column by loading by the HGNC gene list and the previous symbols, then joining
df_gene_info = pd.read_csv(r"../data files/HGNC_gene_info.tsv", sep='\t')
df_prev_sym = pd.read_csv(r"../data files/HGNC_previous_symbols.tsv", sep='\t')
df["HGNC_ID"] = pd.merge(df, df_gene_info, on='Gene_symbol', how='left')[["HGNC_ID_y"]]
df.loc[df["HGNC_ID"].isnull(), "HGNC_ID"] = pd.merge(df, df_prev_sym, left_on='Gene_symbol', right_on='Previous_symbol', how='left')[["HGNC_ID_y"]].squeeze()

# Write the file
df.to_csv(out_filename, sep='\t')